# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [3]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***@Condmaster***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [160]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @Condmaster

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [161]:
#Produto escolhido:
produto = 'Coca Cola'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [162]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [163]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [16]:
#Importa as tabelas do excel
Treino = pd.read_excel('Coca_Cola.xlsx')

In [17]:


#Caracteres indesejados
Indesejados = [":",",",";",".","?","!","/",")","+","…","(","|","^","'","\\","-"]     

contador = 0

#Filtragem dos caracteres indesejados

for i in Treino.Treinamento:
    
    for j in Indesejados:
        i=i.replace(j," ")
   
    Treino.Treinamento[contador]= i
    contador+=1
    

Palavras = []

rel = []

#Palavras indesejadas
remover = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
           'rt','oi',
           'de','da','dos','das','do','de',
           'e','no','na','em','o','as','nas','nos','','é','t','-','https','fm','in','co','eu']
#Palavras como links, nomes de contas
RemoverInteligente = ['@','1','2','3','4','5','6','7','8','9','0']

y=0
x = 0
contador = 0
for i in Treino.Treinamento:
    i = i.split()  #Separa as palavras
    
    while x < len(i):
        j = i[x]
        
        #Remove palavras que não adicionam informação ao Naiv Bayes
        if j in remover:
            i.remove(j)
            x-= 1
            
        #Remove links, nomes das contas do twitter, datas
        for h in RemoverInteligente:
            if h in j:
                i.remove(j)
                x-= 1
                break
        x+=1
        
    x=0
    Palavras += i
    rel += [Treino.Relevancia[y]]*len(i)
    y+=1

In [22]:
#Cria o dicionario que será transformado no dataframe base
TabelaGeral = {
    "Palavras":Palavras,
    "Relevancia":rel
}

#Cria uma lista das palavras relevantes e outra das irrelevantes
Prelevante = []
Pirrelevante = []
i = 0
while i < len(Palavras):
    if rel[i] == 'Relevante':
        Prelevante.append(Palavras[i])
    else:
        Pirrelevante.append(Palavras[i])
    i+=1

In [24]:
#Cria o principal Data Frame com todas palavras
TabelaGeral = pd.DataFrame(TabelaGeral)


#-------------------------------------------------#
#Criação de parametros importantes para análises

#Probabilidade de ser Relevante e de ser Irrelevante
Pr = len(Prelevante)/len(TabelaGeral.Relevancia)
Pi = len(Pirrelevante)/len(TabelaGeral.Relevancia)

#Lista de palavras criada anteriormente será usada para checar se uma palavra existe na base de treinamento

Check = Palavras

#--------------------------------------------------#

In [25]:
def CheckTwit(twit,Pr,Pi,Prelevante,Pirrelevante):
    x=0
    for j in Indesejados:
        twit=twit.replace(j," ")
        
    twit = twit.split()
    
    while x < len(twit):
        j = twit[x]
        
        #Remove palavras que não adicionam informação ao Naiv Bayes
        if j in remover:
            twit.remove(j)
            x-= 1
            
        #Remove links, nomes das contas do twitter, datas
        for h in RemoverInteligente:
            if h in j:
                twit.remove(j)
                x-= 1
                break
        x+=1
    #Contador de palavras desconhecidas    
    n = 0

    #Checagem de palavras desconhecidas
    for i in twit:
        if i not in Prelevante or i not in Pirrelevante:
            n+=1

    TotalRele = 1
    TotalIrrele= 1
    
    #Calculando a probabilidade ser ser relevante ou irrelevante
    if n>0:
        a = 1
        for i in twit:
            PRtwit = (Prelevante.count(i)+a)/(len(Prelevante)+len(TabelaGeral.Palavras.value_counts())+n)
            PItwit = (Pirrelevante.count(i)+a)/(len(Pirrelevante)+len(TabelaGeral.Palavras.value_counts())+n)
            TotalRele = TotalRele*PRtwit
            TotalIrrele = TotalIrrele*PItwit
            
    else:
        for i in twit:
            PRtwit = (Prelevante.count(i))/(len(Prelevante))
            PItwit = (Pirrelevante.count(i))/(len(Pirrelevante))
            
            TotalRele = TotalRele*PRtwit
            TotalIrrele = TotalIrrele*PItwit
            
        
    TotalRele = TotalRele*Pr
    TotalIrrele = TotalIrrele*Pi
    


    if TotalRele > TotalIrrele:
        return 'Relevante'
    else:
        return 'Irrelevante'

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [26]:
#Importa a planilha de teste
Teste = pd.read_excel('Coca_Cola.xlsx',sheetname="Teste")

In [27]:
#Lista que guarda os valores do Naiv Bayes
lista_auxiliar = []

for twit in Teste.Teste:
    lista_auxiliar.append(CheckTwit(twit,Pr,Pi,Prelevante,Pirrelevante))
    
Teste["Relevancia_NaivBayes"]= lista_auxiliar



In [28]:
n = 0
CheckBot= []

for i in Teste.Relevancia_NaivBayes:
    if i == 'Irrelevante' and Teste.Relevancia_Pessoa[n] == 'Irrelevante':
        CheckBot.append("negativos verdadeiros")
        
    elif i == 'Irrelevante' and Teste.Relevancia_Pessoa[n] == 'Relevante':
        CheckBot.append("negativos falsos")
        
    elif i == 'Relevante' and Teste.Relevancia_Pessoa[n] == 'Relevante':
        CheckBot.append("positivos verdadeiros")
        
    elif i == 'Relevante' and Teste.Relevancia_Pessoa[n] == 'Irrelevante':
        CheckBot.append("positivos falsos")
    n+=1

#Transforma em porcentagem
nv = CheckBot.count("negativos verdadeiros")*100/len(CheckBot)
nf = CheckBot.count("negativos falsos")*100/len(CheckBot)
pv = CheckBot.count("positivos verdadeiros")*100/len(CheckBot)
pf = CheckBot.count("positivos falsos")*100/len(CheckBot)

print("Positivos Falsos: {0}%".format(pf))
print("Positivos Verdadeiros: {0}%".format(pv))
print("Negativos Falsos: {0}%".format(nf))
print("Negativos Verdadeiros: {0}%".format(nv))

Positivos Falsos: 23.5%
Positivos Verdadeiros: 14.5%
Negativos Falsos: 6.0%
Negativos Verdadeiros: 56.0%


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


<h1 style="text-align:center;">Analise dos dados obtidos</h1>

<p>  
Analisando os resultados do Naiv-Bayes, pode-se perceber que existem certas peculiaridades. As porcentagens de positivos e negativos verdadeiros somadas estão maiores que os negativos e positivos falsos somados, sendo um ponto positivo para a análise, mostrando que a porcentagem de acerto está maior que 50%, porem pode-se ver uma discrepância entre positivos verdadeiros e negativos verdadeiros,sendo somente 14.5% positivos verdadeiros e 56% negativos verdadeiros. A discrepância citada pode ser fruto da base de treinamento, que possuia 70% de seus tweets como irrelevantes e 30% como relevantes, sendo um dos fatores que criaram tal diferença, não só isso, o Naiv Bayes desenvolvido não analisa a ordem das palavras, somente a probabilidade de elas serem relevantes ou irrelevantes, aumentando o fator erro do classificador. 
</p>

<h2 style="text-align:center;">Erro Cumulativo</h2>

<p>
Algo importante de se notar é que esse classificador não é perfeito, mesmo obtendo um porcentagem de acerto de 70.5%, não podemos fazer a base de dados de treinamento se auto completar com novos tweets. Isso não pode ser feito pois o erro é cumulativo, ou seja, cada vez mais tweets( que foram analisados pelo próprio Naiv Bayes) são colocados na planilha de treinamentos, menos confíaveis serão os resultados, pois resultados errados obtidos pelo Naiv Bayes vão tornar uma parte dos positivos e negativos verdadeiros falsos, e cada vez maior será o erro.
</p>

<h2 style="text-align:center;">Por que usar este classificador?</h2>

<p>
Mesmo não sendo um classificador 100% automatico, ele é muito preciso e consegue ser muito maleável. Ele pode ser usado para diferentes produtos, areas e serviços para analisar diferentes aspectos desejados, como ver críticas ao seu produtos em redes sociais, analisar a formalidade de textos escritos por seus funcíonarios, dentre outras várias aplicações. Trocando a base de treinamento e mudando os filtros dos dados, o classificador consegue ser usado praticamente em qualquer área, com facilidade de ser moldado para quase qualquer necessidade. Além disso, como visto na aplicação acima, as porcentagens de acerto são muito altas(O classificador obteve uma taxa de 70.5% de acerto).
</p>